In [12]:
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import sys
from PIL import Image, ImageDraw, ImageFont
import json
import cv2
import os
import time
import tensorflow as tf
from keras import layers, Input, models, optimizers
from keras.models import Model, Sequential
from keras.models import load_model
from keras import layers, Input, models, optimizers
from keras.layers import MaxPooling2D, Conv2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

In [2]:
# 이전까지 만든 분류 모델 사용할 것
# 새로 crawling 이미지를 가지고 모델을 통해 추천값 matrix 짜기 위한 dataframe 출력
# 동시에 각자 의류 종류, 패턴에 따라 폴더 생성하고 해당되는 이미지 저장 : 나중에 추천시 가져올 수 있게 

In [14]:
# 폴더 생성 함수

def createfolder(foler_dir):
    try:
        if not os.path.exists(folder_dir):
            os.makedirs(folder_dir)
    except OSError:
        print('Error Creating directory'+folder_dir) 

In [15]:
# 자른 이미지 저장할 폴더 생성

shape_list = ['Hoodie', 'Long-sleeve-tee', 'Shirts', 'Short-sleeve-tee', 'Sweater', 
             'Chiffon-skirt', 'Cotten-long-pants', 'Cutoffs', 'H-line-skirt', 'Jeans']
pattern_list = ['Check', 'Dot', 'Floral', 'Graphic', 'Leopard', 'None', 'Stripe']

for shape in shape_list:
    for pattern in pattern_list:
        folder_name = str(shape +'_'+ pattern)
        folder_dir = str('./img/cropped/' + folder_name)
        createfolder(folder_dir)

In [16]:
# 원피스 사진도 혹시 모르니 저장하는 폴더 생성

pattern_list = ['Check', 'Dot', 'Floral', 'Graphic', 'Leopard', 'None', 'Stripe']

for pattern in pattern_list:
    folder_name = str('One-piece' + '_' + pattern)
    folder_dir = str('./img/cropped/' + folder_name)
    createfolder(folder_dir)

In [17]:
# 전신 이미지 가져와서 상하의 원피스 shape, pattern, style 출력 및 각각 사진 잘라서 저장

def style_shape_pattern(img_dir,                                                      # 이미지 가져오는 주소
                        model_pattern, var_pattern, 
                        model_top, var_top, model_bottom, var_bottom, 
                        model_style, var_style,
                        file_name):                                                    # 이미지 잘라서 저장할 떄 사용할 이름 
    
    # VGG 다운
    conv_base = VGG16(weights = 'imagenet',include_top=False,input_shape=(150,150,3))
    
    # top shape model 안잡혀서 한번 더 잡아주고 weight로 로드함
    top_model = models.Sequential()
    top_model.add(layers.Dense(256, activation='relu', input_dim = 4 * 4 * 512))
    top_model.add(layers.Dropout(0.5))
    top_model.add(layers.Dense(4, activation='softmax'))
    top_model.compile(optimizer = optimizers.RMSprop(lr=2e-5),
                      loss='categorical_crossentropy',
                      metrics=['acc'])
    top_model.load_weights(model_top)
    
    # pattern model 안잡혀서 한번 더 잡아주고 weight로 로드함
    pattern_model = models.Sequential()
    pattern_model.add(layers.Dense(256, activation='relu', input_dim = 4 * 4 * 512))
    pattern_model.add(layers.Dropout(0.5))
    pattern_model.add(layers.Dense(7, activation='softmax'))
    pattern_model.compile(optimizer = optimizers.RMSprop(lr=2e-5),
                      loss='categorical_crossentropy',
                      metrics=['acc'])
    pattern_model.load_weights(model_pattern)
    
    # 사용할 model 설정
    model_bottom = load_model(model_bottom)
    model_style = load_model(model_style)
    
    # 카카오 API 설정
    API_URL = 'https://kapi.kakao.com/v1/vision/product/detect'
    MYAPP_KEY = 'd7f9bbf32defc3826369883e77412c40'
    url = "https://kapi.kakao.com/v1/vision/product/detect"       
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}
    files = { 'file' : open(img_dir, 'rb')}
    response = requests.post(url, headers=headers, files=files)
    
    # 카카오 API 저장 및 예외 상황 설정
    info = []
    result = response.json()
    
    if 'result' in result:                                                    
        fig_w, fig_h = result['result']['width'], result['result']['height']
    else:
        return ['NA','NA','NA','NA','NA','NA','NA']

    if 'objects' in result['result']:
        mylist = result['result']['objects']
        cnt = 0
        for objects in mylist:
            if objects['class'] in ['shirts','blouse','t-shirts','pants','outer','skirt','one-piece']:
                cnt = cnt+1
        if cnt > 4:
            return ['NA','NA','NA','NA','NA','NA','NA']
    else:
        return ['NA','NA','NA','NA','NA','NA','NA']
        
    for each in result['result']['objects']:                        
            each = list(each.values())                                            
            info.append(each)
            
    
    # 사진 상하의 구분 및 pattern, shape 예측
    top_pattern = []
    top_shape = []
    bottom_pattern = []
    bottom_shape = []
    one = []
    one_pattern = []
    outer = []
    
    # 사진 전체의 style 예측 
    img = Image.open(img_dir)
    img = img.resize((150,150))
    data = np.array(img)
    data = data.astype('float')
    data = data/255
    data = data.reshape((1,150,150,3))
    st = var_style[model_style.predict_classes(data)[0]]
    style = [st]

    
    # 상의 조금 더 타이트하게 구분하기 위한 crop 값 설정
    upper = list()
    lower = list()
    for _ in result['result']['objects']:
        if _['class'] in ('t-shirts', 'shirts','blouse','sweater'):
            upper = _
        elif _['class'] in ('skirt', 'pants'):
            lower = _
    
    if len(upper) == 0 or len(lower) == 0:
        return ['NA','NA','NA','NA','NA','NA','NA']
    else:
        print(upper, lower)
        x1 = lower['x1']*fig_w -5   #x1
        y1 = lower['y1']*fig_h -5    #y1
        w1 = lower['x2']*fig_w +5   #x2
        h1 = lower['y2']*fig_h +5   #y2

        x2 = upper['x1']*fig_w -5   #x1
        y2 = upper['y1']*fig_h -5   #y1
        w2 = upper['x2']*fig_w +5   #x2
        h2 = y1
    
    # 예측 및 저장하기
    for _ in range(len(info)):
        img = Image.open(img_dir)
        category = info[_][4]

        if category in ('shirts','t-shirts','blouse','sweater'):
            if len(top_pattern) == 1 or len(top_shape) == 1:
                pass
            else:
                x = info[_][0]*fig_w - 5
                y = info[_][1]*fig_h - 5
                w = info[_][2]*fig_w + 5
                h = info[_][3]*fig_h + 5   
                cropped_top = img.crop((x,y,w,h))           # 저장용
                img = Image.open(img_dir)
                img = img.crop((x2,y2,w2,h2))       # 예측용   
                
                img = img.resize((150,150))
                data = np.array(img)
                data = data.astype('float')
                data = data/255
                data = data.reshape((1,150,150,3))  

                # top pattern 예측
                p = conv_base.predict(data)
                p = p.reshape((-1,8192))
                p = pattern_model.predict_classes(p)[0]
                p = var_pattern[p]
                top_pattern.append(p)

                # top shape 예측
                s = conv_base.predict(data)
                s = s.reshape((-1,8192))
                s = top_model.predict_classes(s)[0]
                s = var_top[s]
                top_shape.append(s)  

                
                #자른 사진 저장
                save_name = str(st + '_' + s + '_' + p + '_' + file_name )
                path = str('./img/cropped/' + s + '_' + p + '/')
                cropped_top.save( path + save_name )

        
        elif category in ('pants','skirt'): 
            if len(bottom_pattern) == 1 or len(bottom_shape) == 1:
                pass
            else:            
                x = info[_][0]*fig_w - 5
                y = info[_][1]*fig_h - 5
                w = info[_][2]*fig_w + 5
                h = info[_][3]*fig_h + 5 
                img = img.crop((x,y,w,h))                               
                plt.imshow(img)
                img.show()
                cropped_bottom = img
                
                img = img.resize((150,150))
                data = np.array(img)
                data = data.astype('float')
                data = data/255
                data = data.reshape((1,150,150,3))

                # bottom pattern 예측
                p = folder_list[i].split('_')[1]         
                bottom_pattern.append(p)

                # bottom shape 예측
                s = folder_list[i].split('_')[0]
                bottom_shape.append(s)

                # 자른 이미지 저장
                save_name = str(st + '_' + s + '_' + p + '_' + file_name )
                path = str('./img/cropped/' + s + '_' + p + '/')
                cropped_bottom.save( path + save_name )
                
        # 원피스        
        elif category in ('one-piece'):
            one.append('one-piece')
            
            if len(one_pattern) == 1 :
                pass
            else:
                x = info[_][0]*fig_w - 5
                y = info[_][1]*fig_h - 5
                w = info[_][2]*fig_w + 5
                h = info[_][3]*fig_h + 5
                img = img.crop((x,y,w,h))  
                cropped_one = img
                
                img = img.resize((150,150))
                data = np.array(img)
                data = data.astype('float')
                data = data/255
                data = data.reshape((1,150,150,3))  

                # one-piece pattern 예측
                p = conv_base.predict(data)
                p = p.reshape((-1,8192))
                p = pattern_model.predict_classes(p)[0]
                p = var_pattern[p]
                one_pattern.append(p)

                # 자른 이미지 저장 
                s = 'One-piece'
                save_name = str(st + '_' + s + '_' + p + '_' + file_name )
                path = str('./img/cropped/' + 'One-piece' + '_' + p + '/')
                cropped_one.save( path + save_name )  
                

    # 6)예측값이 없는 경우는 'NA' 값으로 표기
    if len(top_pattern) == 0:
        top_pattern = ['NA']
    if len(top_shape) == 0:
        top_shape = ['NA']        
    if len(bottom_pattern) == 0:
        bottom_pattern = ['NA']
    if len(bottom_shape) == 0:
        bottom_shape = ['NA']
    if len(style) == 0:
        style = ['NA']
    if len(one) == 0:
        one = ['NA']
    if len(one_pattern) == 0:
        one_pattern = ['NA']    


    return ( style + top_shape + top_pattern + bottom_shape + bottom_pattern + one + one_pattern  )


## 하나 폴더 내 여러 이미지 불러와 자르고 분류 및 분류값 dataframe으로 출력

In [18]:
def predict_multi(folder_dir,                                                     
                  model_pattern, var_pattern, 
                  model_top, var_top, model_bottom, var_bottom, 
                  model_style, var_style):
    
    file_list = os.listdir(folder_dir)
    df = pd.DataFrame(index=range(0,0), 
                      columns=['img_dir', 'style', 'top shape', 'top pattern', 'bottom shape', 'bottom pattern',
                               'one-piece','one-piece pattern'])
    display(df)
    
    for _ in range(len(file_list)):
        
        img_dir = str(folder_dir + '/' + file_list[_])
        Image.open(img_dir)
        
        file_name = file_list[_]
        k = style_shape_pattern(img_dir,                                                     
                                model_pattern, var_pattern, 
                                model_top, var_top, model_bottom, var_bottom, 
                                model_style, var_style,
                                file_name)   

        # file_name : crop 하기 전 전신 이미지 파일명
        
        total = [file_name] + k
        df.loc[_] = total
        display(df)
        
        if _ % 100 == 0:
            print(_)
            
    return df


## 코드 실행하기 전에 확인!! : folder_dir, model 유무

In [19]:
model_pattern = './model/pattern_9985.h5'
var_pattern =  ['Check', 'Dot', 'Floral', 'Graphic', 'Leopard', 'None', 'Stripe']

model_top = './model/upper_9978.h5'
var_top = ['Hoodie', 'Long-sleeve-tee', 'Shirts', 'Short-sleeve-tee', 'Sweater',]

model_bottom = './model/lower0425_9990.h5'
var_bottom = ['Chiffon-skirt', 'Cotten-long-pants', 'Cutoffs', 'H-line-skirt', 'Jeans']

model_style = './model/style_model_1_90,63.h5'
var_style =  ['Casual', 'Feminine', 'Formal']

In [ ]:
folder_list = os.listdir('./img/matrix/crawling_bottom')


while True:
    try:
        
        for i in range(len(folder_list)):
            folder_dir = str('./img/matrix/crawling_bottom/' + folder_list[i])



            df = predict_multi(folder_dir,
                               model_pattern, var_pattern, 
                               model_top, var_top, model_bottom, var_bottom, 
                               model_style, var_style)


            df_name = str('matrix_'+ folder_list[i] + '.csv')
            df.to_csv(df_name, header=True, index=False)
            
    except:
        print(img_dir)
        pass

        
        

,img_dir,style,top shape,top pattern,bottom shape,bottom pattern,one-piece,one-piece pattern


{'x1': 0.292553, 'y1': 0.229314, 'x2': 0.647163, 'y2': 0.401891, 'class': 't-shirts'} {'x1': 0.326241, 'y1': 0.37234, 'x2': 0.680851, 'y2': 0.717494, 'class': 'skirt'}


,img_dir,style,top shape,top pattern,bottom shape,bottom pattern,one-piece,one-piece pattern
0,Aa (1).jpg,Feminine,Shirts,Stripe,Chiffon-skirt,Check,NA,NA


0
{'x1': 0.246875, 'y1': 0.29375, 'x2': 0.814063, 'y2': 0.978125, 'class': 't-shirts'} {'x1': 0.29375, 'y1': 0.617188, 'x2': 0.717187, 'y2': 0.939063, 'class': 'skirt'}


,img_dir,style,top shape,top pattern,bottom shape,bottom pattern,one-piece,one-piece pattern
0,Aa (1).jpg,Feminine,Shirts,Stripe,Chiffon-skirt,Check,NA,NA
1,Aa (10).jpg,Formal,Hoodie,Graphic,Chiffon-skirt,Check,NA,NA


{'x1': 0.3125, 'y1': 0.03125, 'x2': 0.675, 'y2': 0.346875, 'class': 't-shirts'} {'x1': 0.382813, 'y1': 0.303125, 'x2': 0.628125, 'y2': 0.842187, 'class': 'skirt'}


,img_dir,style,top shape,top pattern,bottom shape,bottom pattern,one-piece,one-piece pattern
0,Aa (1).jpg,Feminine,Shirts,Stripe,Chiffon-skirt,Check,NA,NA
1,Aa (10).jpg,Formal,Hoodie,Graphic,Chiffon-skirt,Check,NA,NA
2,Aa (11).jpg,Casual,Long-sleeve-tee,None,Chiffon-skirt,Check,NA,NA
